In [31]:
def sum_vec(vec1, vec2):
    for i in range(len(vec1)):
        vec2[i] += vec1[i]
    return vec2


def mul_vec(vec1, vec2):
    vec = [0] * len(vec1)
    for i in range(len(vec1)):
        for j in range(len(vec2)):
            if i + j + 1 < len(vec1):
                vec[(i + j + 1)] += vec1[i] * vec2[j]
    return vec


def mul_vec_num(vec, num):
    for i in range(len(vec)):
        vec[i] *= num
    return vec


def gen_fun_vector_mayerson(gen_funs):
    vector_tree = {}
    for i in range(len(gen_funs)):
        vector_tree[i + 1] = []
        for j in range(1, len(gen_funs[i])):
            if gen_funs[i][j] == 0:
                break
            vector_tree[i + 1].append(gen_funs[i][j] / (j + 1))
    return vector_tree


def gen_fun_vector_positional(gen_funs):
    vector_tree = {}
    for i in range(len(gen_funs)):
        vector_tree[i + 1] = []
        for j in range(1, len(gen_funs[i])):
            if gen_funs[i][j] == 0:
                break
            vector_tree[i + 1].append(gen_funs[i][j] / (j))
    return vector_tree


def gen_fun_avg_tree(gen_fun_tree):
    avg_tree = {}
    for key in gen_fun_tree[0].keys():
        avg_tree[key] = []
        for i in range(1, len(gen_fun_tree[0][key])):
            value = 0
            for trees in gen_fun_tree:
                try:
                    value += trees[key][i]
                except:
                    value += 0
            if value == 0:
                break
            avg_tree[key].append(value / N)
    return avg_tree


def get_sub(x):
    normal = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+-=()"
    sub_s = "ₐ₈CDₑբGₕᵢⱼₖₗₘₙₒₚQᵣₛₜᵤᵥwₓᵧZₐ♭꜀ᑯₑբ₉ₕᵢⱼₖₗₘₙₒₚ૧ᵣₛₜᵤᵥwₓᵧ₂₀₁₂₃₄₅₆₇₈₉₊₋₌₍₎"
    res = x.maketrans("".join(normal), "".join(sub_s))
    return x.translate(res)


def get_super(x):
    normal = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+-=()"
    super_s = "ᴬᴮᶜᴰᴱᶠᴳᴴᴵᴶᴷᴸᴹᴺᴼᴾQᴿˢᵀᵁⱽᵂˣʸᶻᵃᵇᶜᵈᵉᶠᵍʰᶦʲᵏˡᵐⁿᵒᵖ۹ʳˢᵗᵘᵛʷˣʸᶻ⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻⁼⁽⁾"
    res = x.maketrans("".join(normal), "".join(super_s))
    return x.translate(res)


def print_solution(tree, solution_symbol):
    for key in sorted(tree.keys()):
        print(
            solution_symbol
            + get_sub(str(key))
            + " = "
            + str(round(tree[key][0], 4))
            + "r",
            end="",
        )
        for i in range(1, len(tree[key])):
            print(
                " + " + str(round(tree[key][i], 4)) + "r" + get_super(str(i + 1)),
                end="",
            )
        print()


def print_solution_with_r(tree, solution_symbol, r):
    solution_vec = [
        round(sum([tree[i][j] * (r ** (j + 1)) for j in range(len(tree[i]))]), 4)
        for i in sorted(tree.keys())
    ]
    print(solution_symbol + " = " + "(" + "  ".join(list(map(str, solution_vec))) + ")")

def print_top(tree, r, my_dict, top_num):
    solution_vec = [
        round(sum([tree[i][j] * (r ** (j + 1)) for j in range(len(tree[i]))]), 4)
        for i in sorted(tree.keys())
    ]
    sorted_vec = sorted(solution_vec, reverse=True)
    for i in range(top_num):
        print(str(i+1)+')', my_dict[solution_vec.index(sorted_vec[i]) + 1]+': ', sorted_vec[i])
        


class vertex:
    def __init__(self, num):
        self.num = num
        self.l = None
        self.con_ver = []
        self.par = None


class tree:
    def __init__(self, root):
        self.root = root
        self.M = -1

    def new_root(self, ver):
        self.root = ver
        self.root.l = 0
        self.root.par = None
        self.assigning_level(self.root)

    def add_con(self, ver1, ver2):
        ver1.con_ver.append(ver2)
        ver2.con_ver.append(ver1)

    def remove(self, ver):  # Не проверял
        for el in ver.con_ver:
            el.con_ver.remove(ver)
        ver.con_ver = []

    def assigning_level(self, now_el):
        for el in now_el.con_ver:
            if el != now_el.par:
                el.l = now_el.l + 1
                self.M = max(self.M, el.l)
                el.par = now_el
                self.assigning_level(el)

    def generating_function_mayerson(self, ver, flag=1):
        if flag == 1:
            self.M = -1
            self.new_root(ver)
        gen_fun = [1] + [0] * (2 * self.M)
        const_x = [1] + [0] * (2 * self.M)
        gen_fun_sons = []
        for el in ver.con_ver:
            if el != ver.par:
                son_vec = self.generating_function_mayerson(el, flag=0)
                gen_fun_sons.append(son_vec)
                gen_fun = sum_vec(gen_fun, mul_vec(son_vec, const_x))
        if flag == 1:
            for i in range(len(gen_fun_sons)):
                for j in range(len(gen_fun_sons)):
                    if i < j:
                        gen_fun = sum_vec(
                            gen_fun,
                            mul_vec(mul_vec(gen_fun_sons[i], gen_fun_sons[j]), const_x),
                        )
        return gen_fun

    def generating_function_positioal(self, ver, flag=1):
        if flag == 1:
            self.M = -1
            self.new_root(ver)
        gen_fun1 = [1] + [0] * (2 * self.M)
        gen_fun2 = [1] + [0] * (2 * self.M)
        const_x = [1] + [0] * (2 * self.M)
        gen_fun_sons = []
        for el in ver.con_ver:
            if el != ver.par:
                son_vec = self.generating_function_positioal(el, flag=0)
                gen_fun_sons.append(son_vec)
                gen_fun1 = sum_vec(gen_fun1, mul_vec(son_vec[0], const_x))
        if flag == 1:
            for i in range(len(gen_fun_sons)):
                for j in range(len(gen_fun_sons)):
                    if i < j:
                        gen_fun2 = sum_vec(
                            gen_fun2,
                            mul_vec(
                                mul_vec(gen_fun_sons[i][0], gen_fun_sons[j][0]), const_x
                            ),
                        )
            return sum_vec(mul_vec_num(gen_fun1, 0.5), gen_fun2)
        return (gen_fun1, gen_fun2)

    def generating_function_avg_tree(self, ver, flag=1, gen_fun_tree={}):
        if flag == 1:
            self.M = -1
            self.new_root(ver)
            gen_fun_tree = {}
        gen_fun1 = [1] + [0] * (2 * self.M)
        gen_fun2 = [1] + [0] * (2 * self.M)
        const_x = [1] + [0] * (2 * self.M)
        gen_fun_sons = []
        for el in ver.con_ver:
            if el != ver.par:
                son_vec = self.generating_function_avg_tree(
                    el, flag=0, gen_fun_tree=gen_fun_tree
                )
                gen_fun_sons.append(son_vec)
                gen_fun1 = sum_vec(gen_fun1, mul_vec(son_vec[0], const_x))
                gen_fun2 = gen_fun1
        for i in range(len(gen_fun_sons)):
            for j in range(len(gen_fun_sons)):
                if i < j:
                    gen_fun2 = sum_vec(
                        gen_fun2,
                        mul_vec(
                            mul_vec(gen_fun_sons[i][0], gen_fun_sons[j][0]), const_x
                        ),
                    )
        gen_fun_tree[ver.num] = gen_fun2
        if flag == 1:
            return gen_fun_tree
        return (gen_fun1, gen_fun2)

    
    
    
class graph_vertex:
    def __init__(self, num):
        self.num = num
        self.con_ver = []  
    
class graph:
    def add_con(self, ver1, ver2):
        ver1.con_ver.append(ver2)
        ver2.con_ver.append(ver1)

    def remove(self, ver):  # Не проверял
        for el in ver.con_ver:
            el.con_ver.remove(ver)
        ver.con_ver = []
    def mayersons_vector_additional(self, ver, m_vec, way=[], l = 0):
        my_way = way.copy()
        my_way.append(ver.num)
        l += 1
        for num in my_way:
            if l == 1:
                m_vec[num][l-1] += 1
            else:
                m_vec[num][l-1] += 1/(2*l)
        #print(my_way, l)
        for v in ver.con_ver:
            if v.num not in my_way:
                self.mayersons_vector_additional(v, m_vec, way=my_way, l=l)
        return
                
    def mayersons_vector(self, ver, m_vec):
        for i in range(1, len(ver)):
            self.mayersons_vector_additional(ver[i], m_vec, way=[])
    def positioal_vector_additional(self, ver, m_vec, way=[], l = 0):
        my_way = way.copy()
        my_way.append(ver.num)
        l += 1
        for num in my_way:
            if l == 1:
                m_vec[num][l-1] += 1
            else:
                if num == my_way[0] or num == my_way[-1]:
                    m_vec[num][l-1] += 1/(2*2*(l-1))
                else:
                    m_vec[num][l-1] += 1/(2*(l-1))
        #print(my_way, l)
        for v in ver.con_ver:
            if v.num not in my_way:
                self.positioal_vector_additional(v, m_vec, way=my_way, l=l)
        return
    def positioal_vector(self, ver, m_vec):
        for i in range(1, len(ver)):
            self.positioal_vector_additional(ver[i], m_vec, way=[])
    
    def avg_tree_vector_additional(self, ver, num=1, span_tree_ver=[], span_tree_con=[], del_con=[], span_trees=[], past_num=[1]):
        #print(span_tree_con)
        now_ver = ver[num]
        if num not in span_tree_ver:
            span_tree_ver.append(num)
        #print(span_tree_con, ' | ', del_con, ' | ', past_num, ' | ', span_trees if len(span_trees) == 0 else span_trees[-1])
        if len(span_tree_ver) == len(ver)-1:
            if span_tree_con not in span_trees:
                #print(len(span_trees))
                span_trees.append(span_tree_con)
            return
        for v in now_ver.con_ver:
            if v.num in span_tree_ver or [now_ver.num, v.num] in del_con or [v.num, now_ver.num] in del_con:
                continue
            else:
                if len(now_ver.con_ver) > 1:
                    self.avg_tree_vector_additional(ver, 
                                                     num=now_ver.num, 
                                                     span_tree_ver=span_tree_ver.copy(), 
                                                     span_tree_con=span_tree_con.copy(), 
                                                     del_con=del_con+[[now_ver.num, v.num]], 
                                                     span_trees=span_trees,
                                                     past_num=past_num.copy())
                
                self.avg_tree_vector_additional(ver, 
                                                 num=now_ver.num, 
                                                 span_tree_ver=span_tree_ver + [v.num], 
                                                 span_tree_con=span_tree_con + [[now_ver.num, v.num]], 
                                                 del_con=del_con.copy(),
                                                 span_trees=span_trees,
                                                 past_num=past_num.copy())
                return
        for n in span_tree_ver:
            if n not in past_num:
                self.avg_tree_vector_additional(ver, 
                                                 num=n, 
                                                 span_tree_ver=span_tree_ver.copy(), 
                                                 span_tree_con=span_tree_con.copy(), 
                                                 del_con=del_con.copy(),
                                                 span_trees=span_trees,
                                                 past_num=past_num+[n])
                break
        return
    
    def non_del_cons(self, ver):
        non_del_vec = []
        non_del_con = []
        for v in ver:
            if len(v.con_ver) == 1:
                non_del_vec.append(v)
        for v in non_del_vec:
            for vers in v.con_ver:
                if vers not in non_del_vec:
                    next_v = vers
            non_del_con.append([v.num, next_v.num])
            if len(next_v.con_ver) == 2:
                non_del_vec.append(next_v)
        return non_del_vec, non_del_con
            
    
    def avg_tree_vector(self, ver):
        span_trees=[]
        past_num_ver, span_tree_con = self.non_del_cons(ver)
        past_num = [v.num for v in past_num_ver]
        num = 1
        for v in ver:
            if v.num not in past_num:
                num = v.num
        self.avg_tree_vector_additional(ver, 
                                        num=num, 
                                        span_tree_ver=past_num.copy(), 
                                        span_tree_con=span_tree_con.copy(), 
                                        del_con=[], 
                                        span_trees=span_trees, 
                                        past_num=past_num.copy())
#         for i in range(len(span_trees)):
#             for j in range(len(span_trees[i])):
#                 if span_trees[i][j][0] > span_trees[i][j][1]:
#                     print(span_trees[i][j][0], span_trees[i][j][1])
#                     span_trees[i][j] = [span_trees[i][j][1], span_trees[i][j][0]]
#                     print(span_trees[i][j])
#                     print()
                
        check_span_tree = [set([','.join(map(str, con)) for con in trees])  for trees in span_trees]
        final_check_span_tree = []
        for trees in check_span_tree:
            if trees not in final_check_span_tree:
                #print(trees, len(final_check_span_tree))
                final_check_span_tree.append(trees)
        final_span_tree = [[list(map(int, con.split(','))) for con in tr]  for tr in final_check_span_tree]
        #print(len(final_span_tree), 'wfefgoqijewiurgohqweiuorgh')
        avg_tree_sol = {}
        for i in range(1, len(ver)):
            avg_tree_sol[i] = [0]*len(ver)
        for trees in final_span_tree:
            vers = [vertex(j) for j in range(len(ver))]
            Tree_atv = tree(vers[1])
            for con in trees:
                Tree_atv.add_con(vers[con[0]], vers[con[1]])
            Tree_atv.new_root(Tree_atv.root)  
            #print(trees)
            sol_tree = gen_fun_avg_tree(
                [Tree_atv.generating_function_avg_tree(vers[i]) for i in range(1, len(vers))]
            )
            #print(sol_tree)
            for n in avg_tree_sol.keys():
                for i in range(min(len(avg_tree_sol[n]), len(sol_tree[n]))):
                    avg_tree_sol[n][i] += sol_tree[n][i]/len(final_span_tree)
        return avg_tree_sol
        

In [2]:
with open("Tree_for_diploma.txt", "r") as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if i == 0:
            N = int(lines[i].strip())
            ver = [vertex(j) for j in range(N+1)]
            Tree = tree(ver[1])
        else:
            k, l = map(int, lines[i].split())
            Tree.add_con(ver[k], ver[l])
    Tree.new_root(Tree.root)


In [3]:
print("Вектор Майерсона:")
sol_tree = gen_fun_vector_mayerson(
    [Tree.generating_function_mayerson(ver[i]) for i in range(1, N + 1)]
)
print_solution(sol_tree, "Y")
print()

print("Позиционное решение:")
sol_tree = gen_fun_vector_positional(
    [Tree.generating_function_positioal(ver[i]) for i in range(1, N + 1)]
)
print_solution(sol_tree, "\u03C0")
print()

print("Среднее решение дерева:")
sol_tree = gen_fun_avg_tree(
    [Tree.generating_function_avg_tree(ver[i]) for i in range(1, N + 1)]
)
print_solution(sol_tree, "AT")

Вектор Майерсона:
Y₁ = 1.0r + 1.3333r² + 1.25r³ + 0.8r⁴ + 0.3333r⁵
Y₂ = 1.5r + 2.0r² + 1.75r³ + 0.8r⁴ + 0.3333r⁵
Y₃ = 1.0r + 0.6667r² + 0.75r³ + 0.8r⁴ + 0.3333r⁵
Y₄ = 1.5r + 1.6667r² + 1.25r³ + 0.6r⁴ + 0.3333r⁵
Y₅ = 0.5r + 0.6667r² + 0.75r³ + 0.2r⁴
Y₆ = 0.5r + 0.3333r² + 0.25r³ + 0.4r⁴ + 0.3333r⁵
Y₇ = 0.5r + 0.6667r² + 0.5r³ + 0.2r⁴ + 0.1667r⁵
Y₈ = 0.5r + 0.6667r² + 0.5r³ + 0.2r⁴ + 0.1667r⁵

Позиционное решение:
π₁ = 1.0r + 1.25r² + 1.3333r³ + 1.0r⁴ + 0.4r⁵
π₂ = 1.5r + 2.25r² + 2.1667r³ + 1.0r⁴ + 0.4r⁵
π₃ = 1.0r + 0.75r² + 0.6667r³ + 0.75r⁴ + 0.4r⁵
π₄ = 1.5r + 2.0r² + 1.5r³ + 0.625r⁴ + 0.4r⁵
π₅ = 0.5r + 0.5r² + 0.5r³ + 0.125r⁴
π₆ = 0.5r + 0.25r² + 0.1667r³ + 0.25r⁴ + 0.2r⁵
π₇ = 0.5r + 0.5r² + 0.3333r³ + 0.125r⁴ + 0.1r⁵
π₈ = 0.5r + 0.5r² + 0.3333r³ + 0.125r⁴ + 0.1r⁵

Среднее решение дерева:
AT₁ = 1.125r + 1.625r² + 1.125r³ + 0.5r⁴ + 0.25r⁵
AT₂ = 2.125r + 3.125r² + 2.875r³ + 1.25r⁴ + 0.5r⁵
AT₃ = 1.125r + 0.375r² + 0.625r³ + 0.75r⁴ + 0.25r⁵
AT₄ = 2.125r + 2.0r² + 1.375r³ + 0.875r⁴ + 0.5r⁵

In [5]:
print(
    "Задайте коэффициент уменьшения дохода(r) от увеличения длины связи в диапозоне [0,1]:"
)
r = float(input())
while r > 1 or r < 0:
    print("Невозможное значение r. Задайте r в диапозоне [0,1]:")
    r = float(input())
print()

print("Решения для заданного r:")

print("Вектор Майерсона:")
sol_tree = gen_fun_vector_mayerson(
    [Tree.generating_function_mayerson(ver[i]) for i in range(1, N + 1)]
)
print_solution_with_r(sol_tree, "Y", r)
print()

print("Позиционное решение:")
sol_tree = gen_fun_vector_positional(
    [Tree.generating_function_positioal(ver[i]) for i in range(1, N + 1)]
)
print_solution_with_r(sol_tree, "\u03C0", r)
print()

print("Среднее решение дерева:")
sol_tree = gen_fun_avg_tree(
    [Tree.generating_function_avg_tree(ver[i]) for i in range(1, N + 1)]
)
print_solution_with_r(sol_tree, "AT", r)

Задайте коэффициент уменьшения дохода(r) от увеличения длины связи в диапозоне [0,1]:
1

Решения для заданного r:
Вектор Майерсона:
Y = (4.7167  6.3833  3.55  5.35  2.1167  1.8167  2.0333  2.0333)

Позиционное решение:
π = (4.9833  7.3167  3.5667  6.025  1.625  1.3667  1.5583  1.5583)

Среднее решение дерева:
AT = (4.625  9.875  3.125  6.875  0.875  0.875  0.875  0.875)


In [6]:
with open("Tree_for_diploma2.txt", "r") as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if i == 0:
            N = int(lines[i].strip())
            ver = [vertex(j) for j in range(N + 1)]
            Tree = tree(ver[1])
        else:
            k, l = map(int, lines[i].split())
            Tree.add_con(ver[k], ver[l])
    Tree.new_root(Tree.root)

In [7]:
print("Вектор Майерсона:")
sol_tree = gen_fun_vector_mayerson(
    [Tree.generating_function_mayerson(ver[i]) for i in range(1, N + 1)]
)
print_solution(sol_tree, "Y")
print()

print("Позиционное решение:")
sol_tree = gen_fun_vector_positional(
    [Tree.generating_function_positioal(ver[i]) for i in range(1, N + 1)]
)
print_solution(sol_tree, "\u03C0")
print()

print("Среднее решение дерева:")
sol_tree = gen_fun_avg_tree(
    [Tree.generating_function_avg_tree(ver[i]) for i in range(1, N + 1)]
)
print_solution(sol_tree, "AT")

Вектор Майерсона:
Y₁ = 0.5r + 0.6667r² + 0.25r³
Y₂ = 1.5r + 1.3333r² + 0.5r³
Y₃ = 1.0r + 1.0r² + 0.5r³
Y₄ = 0.5r + 0.6667r² + 0.25r³
Y₅ = 0.5r + 0.3333r² + 0.5r³

Позиционное решение:
π₁ = 0.5r + 0.5r² + 0.1667r³
π₂ = 1.5r + 1.75r² + 0.6667r³
π₃ = 1.0r + 1.0r² + 0.6667r³
π₄ = 0.5r + 0.5r² + 0.1667r³
π₅ = 0.5r + 0.25r² + 0.3333r³

Среднее решение дерева:
AT₁ = 0.2r + 0.4r² + 0.2r³
AT₂ = 2.2r + 2.0r² + 0.8r³
AT₃ = 1.2r + 1.0r² + 0.4r³
AT₄ = 0.2r + 0.4r² + 0.2r³
AT₅ = 0.2r + 0.2r² + 0.4r³


In [8]:
print(
    "Задайте коэффициент уменьшения дохода(r) от увеличения длины связи в диапозоне [0,1]:"
)
r = float(input())
while r > 1 or r < 0:
    print("Невозможное значение r. Задайте r в диапозоне [0,1]:")
    r = float(input())
print()

print("Решения для заданного r:")

print("Вектор Майерсона:")
sol_tree = gen_fun_vector_mayerson(
    [Tree.generating_function_mayerson(ver[i]) for i in range(1, N + 1)]
)
print_solution_with_r(sol_tree, "Y", r)
print()

print("Позиционное решение:")
sol_tree = gen_fun_vector_positional(
    [Tree.generating_function_positioal(ver[i]) for i in range(1, N + 1)]
)
print_solution_with_r(sol_tree, "\u03C0", r)
print()

print("Среднее решение дерева:")
sol_tree = gen_fun_avg_tree(
    [Tree.generating_function_avg_tree(ver[i]) for i in range(1, N + 1)]
)
print_solution_with_r(sol_tree, "AT", r)

Задайте коэффициент уменьшения дохода(r) от увеличения длины связи в диапозоне [0,1]:
1

Решения для заданного r:
Вектор Майерсона:
Y = (1.4167  3.3333  2.5  1.4167  1.3333)

Позиционное решение:
π = (1.1667  3.9167  2.6667  1.1667  1.0833)

Среднее решение дерева:
AT = (0.8  5.0  2.6  0.8  0.8)


In [9]:
with open("graph_for_diploma.txt", "r") as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if i == 0:
            N = int(lines[i].strip())
            ver = [graph_vertex(j) for j in range(N+1)]
            my_graph = graph()
        else:
            k, l = map(int, lines[i].split())
            my_graph.add_con(ver[k], ver[l])

In [10]:
#Вектор Майерсона:
m_vec = {}
for n in range(1, N+1):
    m_vec[n] = [0]*N
my_graph.mayersons_vector(ver, m_vec=m_vec)
for key in m_vec.keys():
    m_vec[key].pop(0)
    
#Позиционное значение:
p_vec = {}
for n in range(1, N+1):
    p_vec[n] = [0]*N
my_graph.positioal_vector(ver, m_vec=p_vec)
for key in p_vec.keys():
    p_vec[key].pop(0)
    
#Среднее решение дерева:
num_ways = {}
for i in range(len(m_vec[1])):
    num_ways[i] = 0
    for key in m_vec.keys():
        num_ways[i] += m_vec[key][i]

t_vec = {}
for n in range(1, N+1):
    t_vec[n] = [0]*N
t_vec = my_graph.avg_tree_vector(ver)
t_vec_sum_i = {}

for i in range(len(m_vec[1])):
    t_vec_sum_i[i] = 0
    for key in t_vec.keys():
        t_vec_sum_i[i] += t_vec[key][i]

norm_t_vec = {}
for key in t_vec.keys():   
    norm_t_vec[key] = []
    for i in range(len(m_vec[1])):
        norm_t_vec[key].append(t_vec[key][i]/t_vec_sum_i[i]*num_ways[i])

In [11]:
print("Вектор Майерсона:")
print_solution(m_vec, "Y")
print()

print("Позиционное значение:")
print_solution(p_vec, "\u03C0")
print()

print("Среднее решение дерева:")
print_solution(norm_t_vec, "AT")

Вектор Майерсона:
Y₁ = 0.5r + 0.6667r² + 0.5r³
Y₂ = 1.5r + 1.6667r² + 0.5r³
Y₃ = 1.0r + 1.3333r² + 0.5r³
Y₄ = 1.0r + 1.3333r² + 0.5r³

Позиционное значение:
π₁ = 0.5r + 0.5r² + 0.3333r³
π₂ = 1.5r + 2.0r² + 0.6667r³
π₃ = 1.0r + 1.25r² + 0.5r³
π₄ = 1.0r + 1.25r² + 0.5r³

Среднее решение дерева:
AT₁ = 0.3333r + 0.7143r² + 0.5r³
AT₂ = 2.1111r + 2.1429r² + 0.5r³
AT₃ = 0.7778r + 1.0714r² + 0.5r³
AT₄ = 0.7778r + 1.0714r² + 0.5r³


In [12]:
print(
    "Задайте коэффициент уменьшения дохода(r) от увеличения длины связи в диапозоне [0,1]:"
)
r = float(input())
while r > 1 or r < 0:
    print("Невозможное значение r. Задайте r в диапозоне [0,1]:")
    r = float(input())
print()

print("Решения для заданного r:")

print("Вектор Майерсона:")
print_solution_with_r(m_vec, "Y", r)
print()

print("Позиционное решение:")
print_solution_with_r(p_vec, "\u03C0", r)
print()

print("Среднее решение дерева:")
print_solution_with_r(norm_t_vec, "AT", r)

Задайте коэффициент уменьшения дохода(r) от увеличения длины связи в диапозоне [0,1]:
1

Решения для заданного r:
Вектор Майерсона:
Y = (1.6667  3.6667  2.8333  2.8333)

Позиционное решение:
π = (1.3333  4.1667  2.75  2.75)

Среднее решение дерева:
AT = (1.5476  4.754  2.3492  2.3492)


In [13]:
with open("graph_for_diploma_2.txt", "r") as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if i == 0:
            N = int(lines[i].strip())
            ver = [graph_vertex(j) for j in range(N+1)]
            my_graph = graph()
        else:
            k, l = map(int, lines[i].split())
            my_graph.add_con(ver[k], ver[l])

In [14]:
#Вектор Майерсона:
m_vec = {}
for n in range(1, N+1):
    m_vec[n] = [0]*N
my_graph.mayersons_vector(ver, m_vec=m_vec)
for key in m_vec.keys():
    m_vec[key].pop(0)
    
#Позиционное значение:
p_vec = {}
for n in range(1, N+1):
    p_vec[n] = [0]*N
my_graph.positioal_vector(ver, m_vec=p_vec)
for key in p_vec.keys():
    p_vec[key].pop(0)
    
#Среднее решение дерева:
num_ways = {}
for i in range(len(m_vec[1])):
    num_ways[i] = 0
    for key in m_vec.keys():
        num_ways[i] += m_vec[key][i]

t_vec = {}
for n in range(1, N+1):
    t_vec[n] = [0]*N
t_vec = my_graph.avg_tree_vector(ver)
t_vec_sum_i = {}

for i in range(len(m_vec[1])):
    t_vec_sum_i[i] = 0
    for key in t_vec.keys():
        t_vec_sum_i[i] += t_vec[key][i]

norm_t_vec = {}
for key in t_vec.keys():   
    norm_t_vec[key] = []
    for i in range(len(m_vec[1])):
        norm_t_vec[key].append(t_vec[key][i]/t_vec_sum_i[i]*num_ways[i])

In [15]:
print("Вектор Майерсона:")
print_solution(m_vec, "Y")
print()

print("Позиционное значение:")
print_solution(p_vec, "\u03C0")
print()

print("Среднее решение дерева:")
print_solution(norm_t_vec, "AT")

Вектор Майерсона:
Y₁ = 1.0r + 1.6667r² + 1.5r³ + 0.8r⁴
Y₂ = 2.0r + 3.3333r² + 2.0r³ + 0.8r⁴
Y₃ = 1.0r + 1.6667r² + 1.5r³ + 0.8r⁴
Y₄ = 1.0r + 1.6667r² + 1.5r³ + 0.8r⁴
Y₅ = 1.0r + 1.6667r² + 1.5r³ + 0.8r⁴

Позиционное значение:
π₁ = 1.0r + 1.5r² + 1.3333r³ + 0.75r⁴
π₂ = 2.0r + 4.0r² + 2.6667r³ + 1.0r⁴
π₃ = 1.0r + 1.5r² + 1.3333r³ + 0.75r⁴
π₄ = 1.0r + 1.5r² + 1.3333r³ + 0.75r⁴
π₅ = 1.0r + 1.5r² + 1.3333r³ + 0.75r⁴

Среднее решение дерева:
AT₁ = 0.8r + 1.2353r² + 1.4r³ + 0.8r⁴
AT₂ = 2.8r + 5.0588r² + 2.4r³ + 0.8r⁴
AT₃ = 0.8r + 1.2353r² + 1.4r³ + 0.8r⁴
AT₄ = 0.8r + 1.2353r² + 1.4r³ + 0.8r⁴
AT₅ = 0.8r + 1.2353r² + 1.4r³ + 0.8r⁴


In [16]:
print(
    "Задайте коэффициент уменьшения дохода(r) от увеличения длины связи в диапозоне [0,1]:"
)
r = float(input())
while r > 1 or r < 0:
    print("Невозможное значение r. Задайте r в диапозоне [0,1]:")
    r = float(input())
print()

print("Решения для заданного r:")

print("Вектор Майерсона:")
print_solution_with_r(m_vec, "Y", r)
print()

print("Позиционное решение:")
print_solution_with_r(p_vec, "\u03C0", r)
print()

print("Среднее решение дерева:")
print_solution_with_r(norm_t_vec, "AT", r)

Задайте коэффициент уменьшения дохода(r) от увеличения длины связи в диапозоне [0,1]:
1

Решения для заданного r:
Вектор Майерсона:
Y = (4.9667  8.1333  4.9667  4.9667  4.9667)

Позиционное решение:
π = (4.5833  9.6667  4.5833  4.5833  4.5833)

Среднее решение дерева:
AT = (4.2353  11.0588  4.2353  4.2353  4.2353)


In [17]:
dictionary_metro = {
    1: 'Проспект Ветеранов',
    2: 'Ленинский проспект',
    3: 'Автово',
    4: 'Кировский завод',
    5: 'Нарвская',
    6: 'Балтийская',
    7: 'Технологический институт(1,2)', 
    8: 'Пушкинская (Звенигородская)',
    9: 'Владимирская (Достоевская)',
    10: 'Площадь Восстания (Маяковская)',
    11: 'Чернышевская',
    12: 'Площадь Ленина',
    13: 'Выборгская',
    14: 'Лесная',
    15: 'Площадь Мужества',
    16: 'Политехническая',
    17: 'Академическая',
    18: 'Гражданский проспект',
    19: 'Девяткино',
    20: 'Купчино',
    21: 'Звёздная',
    22: 'Московская',
    23: 'Парк Победы',
    24: 'Электросила',
    25: 'Московские ворота',
    26: 'Фрунзенская',
    27: 'Сенная площадь (Спасская/Садовая)',
    28: 'Невский проспект (Гостиный двор)',
    29: 'Горьковская',
    30: 'Петроградская',
    31: 'Чёрная речка',
    32: 'Пионерская',
    33: 'Удельная',
    34: 'Озерки',
    35: 'Проспект Просвящения',
    36: 'Парнас',
    37: 'Шушары',
    38: 'Дунайская',
    39: 'Проспект Славы',
    40: 'Международная',
    41: 'Бухарестская',
    42: 'Волковская',
    43: 'Обводный канал',
    44: 'Адмиралтейская',
    45: 'Спортивная',
    46: 'Чкаловская',
    47: 'Крестовский остров',
    48: 'Старая деревня',
    49: 'Комендантский проспект',
    50: 'Рыбацкое',
    51: 'Обухово',
    52: 'Пролетарская',
    53: 'Ломоносовская',
    54: 'Елизаровская',
    55: 'Пл. Александра Невского (1,2)',
    56: 'Василеостровская',
    57: 'Приморская',
    58: 'Зенит',
    59: 'Беговая',
    60: 'Улица Дыбенко',
    61: 'Проспект Большевиков',
    62: 'Ладожская',
    63: 'Новочеркасская',
    64: 'Лиговский проспект'
}

In [18]:
with open("metro_spb.txt", "r") as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if i == 0:
            N = int(lines[i].strip())
            ver = [graph_vertex(j) for j in range(N+1)]
            my_graph = graph()
        else:
            k, l = map(int, lines[i].split())
            my_graph.add_con(ver[k], ver[l])

In [19]:
#Вектор Майерсона:
m_vec = {}
for n in range(1, N+1):
    m_vec[n] = [0]*N
my_graph.mayersons_vector(ver, m_vec=m_vec)
for key in m_vec.keys():
    m_vec[key].pop(0)
    
#Позиционное значение:
p_vec = {}
for n in range(1, N+1):
    p_vec[n] = [0]*N
my_graph.positioal_vector(ver, m_vec=p_vec)
for key in p_vec.keys():
    p_vec[key].pop(0)

#Среднее решение дерева:
num_ways = {}
for i in range(len(m_vec[1])):
    num_ways[i] = 0
    for key in m_vec.keys():
        num_ways[i] += m_vec[key][i]

t_vec = {}
for n in range(1, N+1):
    t_vec[n] = [0]*N
t_vec = my_graph.avg_tree_vector(ver)
t_vec_sum_i = {}

for i in range(len(m_vec[1])):
    t_vec_sum_i[i] = 0
    for key in t_vec.keys():
        t_vec_sum_i[i] += t_vec[key][i]

norm_t_vec = {}
for key in t_vec.keys():   
    norm_t_vec[key] = []
    for i in range(len(m_vec[1])):
        if t_vec_sum_i[i] == 0:
            break
        norm_t_vec[key].append(t_vec[key][i]/t_vec_sum_i[i]*num_ways[i])

In [20]:
print("Вектор Майерсона:")
print_solution(m_vec, "Y")
print()

print("Позиционное значение:")
print_solution(p_vec, "\u03C0")
print()

print("Среднее решение дерева:")
print_solution(norm_t_vec, "AT")

Вектор Майерсона:
Y₁ = 0.5r + 0.3333r² + 0.25r³ + 0.2r⁴ + 0.1667r⁵ + 0.1429r⁶ + 0.375r⁷ + 0.8889r⁸ + 1.7r⁹ + 2.5455r¹⁰ + 4.1667r¹¹ + 4.8462r¹² + 4.8571r¹³ + 4.4r¹⁴ + 3.5r¹⁵ + 2.4706r¹⁶ + 1.7222r¹⁷ + 1.2632r¹⁸ + 0.8r¹⁹ + 0.4762r²⁰ + 0.1818r²¹ + 0r²² + 0r²³ + 0r²⁴ + 0r²⁵ + 0r²⁶ + 0r²⁷ + 0r²⁸ + 0r²⁹ + 0r³⁰ + 0r³¹ + 0r³² + 0r³³ + 0r³⁴ + 0r³⁵ + 0r³⁶ + 0r³⁷ + 0r³⁸ + 0r³⁹ + 0r⁴⁰ + 0r⁴¹ + 0r⁴² + 0r⁴³ + 0r⁴⁴ + 0r⁴⁵ + 0r⁴⁶ + 0r⁴⁷ + 0r⁴⁸ + 0r⁴⁹ + 0r⁵⁰ + 0r⁵¹ + 0r⁵² + 0r⁵³ + 0r⁵⁴ + 0r⁵⁵ + 0r⁵⁶ + 0r⁵⁷ + 0r⁵⁸ + 0r⁵⁹ + 0r⁶⁰ + 0r⁶¹ + 0r⁶² + 0r⁶³
Y₂ = 1.0r + 0.6667r² + 0.5r³ + 0.4r⁴ + 0.3333r⁵ + 0.5714r⁶ + 1.375r⁷ + 2.7778r⁸ + 4.5r⁹ + 7.0909r¹⁰ + 9.4167r¹¹ + 10.0769r¹² + 9.5714r¹³ + 8.1333r¹⁴ + 6.125r¹⁵ + 4.2941r¹⁶ + 3.0556r¹⁷ + 2.1053r¹⁸ + 1.3r¹⁹ + 0.6667r²⁰ + 0.1818r²¹ + 0r²² + 0r²³ + 0r²⁴ + 0r²⁵ + 0r²⁶ + 0r²⁷ + 0r²⁸ + 0r²⁹ + 0r³⁰ + 0r³¹ + 0r³² + 0r³³ + 0r³⁴ + 0r³⁵ + 0r³⁶ + 0r³⁷ + 0r³⁸ + 0r³⁹ + 0r⁴⁰ + 0r⁴¹ + 0r⁴² + 0r⁴³ + 0r⁴⁴ + 0r⁴⁵ + 0r⁴⁶ + 0r⁴⁷ + 0r⁴⁸ + 0r⁴⁹ + 0r⁵⁰ + 0r⁵¹ + 0r⁵² + 0r⁵³ + 0r⁵⁴ + 0r

In [22]:
print(
    "Задайте коэффициент уменьшения дохода(r) от увеличения длины связи в диапозоне [0,1]:"
)
r = float(input())
while r > 1 or r < 0:
    print("Невозможное значение r. Задайте r в диапозоне [0,1]:")
    r = float(input())
print()

print("Решения для заданного r:")
print()

print("Вектор Майерсона:")
print_solution_with_r(m_vec, "Y", r)
print()

print("Позиционное решение:")
print_solution_with_r(p_vec, "\u03C0", r)
print()

print("Среднее решение дерева:")
print_solution_with_r(norm_t_vec, "AT", r)

Задайте коэффициент уменьшения дохода(r) от увеличения длины связи в диапозоне [0,1]:
0.5

Решения для заданного r:
Вектор Майерсона:
Y = (0.401  0.8198  1.0293  1.2493  1.6616  2.616  7.6398  8.853  9.8006  8.8115  2.6692  1.7012  1.3007  1.1265  1.0429  0.9881  0.9202  0.7771  0.3877  0.393  0.7939  0.9628  1.0888  1.2775  1.6772  2.6259  11.6103  8.0778  2.5767  1.6599  1.2788  1.1086  1.0165  0.9323  0.7819  0.3892  0.3939  0.7968  0.97  1.106  1.3176  1.7712  2.8515  2.9683  1.8001  1.3054  1.0519  0.8285  0.4036  0.4121  0.8564  1.125  1.4866  2.2542  6.3403  2.4919  1.5201  1.005  0.457  0.4445  0.9635  1.4094  2.2105  4.0576)

Позиционное решение:
π = (0.3547  0.8243  1.015  1.2017  1.5543  2.4044  8.0061  9.2996  10.4252  9.395  2.4559  1.5913  1.2483  1.1028  1.0339  0.9883  0.929  0.793  0.3474  0.3502  0.8052  0.9622  1.0696  1.2276  1.5691  2.4144  12.7497  8.5872  2.3757  1.5562  1.2299  1.0876  1.0111  0.9384  0.7964  0.3482  0.3507  0.8073  0.9679  1.0834  1.2607  1.648

In [34]:
print(
    "Задайте коэффициент уменьшения дохода(r) от увеличения длины связи в диапозоне [0,1]:"
)
r = float(input())
while r > 1 or r < 0:
    print("Невозможное значение r. Задайте r в диапозоне [0,1]:")
    r = float(input())
print()

print(
    "Задайте количество элементов в топе:"
)
top_n = int(input())
while top_n > len(ver)-1 or top_n < 0:
    print("Невозможное значение количества элементов в топе. Задайте его в диапозоне [0,количество вершин в графе]:")
    top_n = float(input())
print()

print("Решения для заданного r и количества элементов в топе:")
print()

print("Вектор Майерсона:")

print_top(m_vec, 0.5, dictionary_metro, 10)
print()

print("Позиционное решение:")
print_top(p_vec, 0.5, dictionary_metro, 10)
print()

print("Среднее решение дерева:")
print_top(norm_t_vec, 0.5, dictionary_metro, 10)

Задайте коэффициент уменьшения дохода(r) от увеличения длины связи в диапозоне [0,1]:
0.5

Задайте количество элементов в топе:
10

Решения для заданного r и количества элементов в топе:

Вектор Майерсона:
1) Сенная площадь (Спасская/Садовая):  11.6103
2) Владимирская (Достоевская):  9.8006
3) Пушкинская (Звенигородская):  8.853
4) Площадь Восстания (Маяковская):  8.8115
5) Невский проспект (Гостиный двор):  8.0778
6) Технологический институт(1,2):  7.6398
7) Пл. Александра Невского (1,2):  6.3403
8) Лиговский проспект:  4.0576
9) Адмиралтейская:  2.9683
10) Обводный канал:  2.8515

Позиционное решение:
1) Сенная площадь (Спасская/Садовая):  12.7497
2) Владимирская (Достоевская):  10.4252
3) Площадь Восстания (Маяковская):  9.395
4) Пушкинская (Звенигородская):  9.2996
5) Невский проспект (Гостиный двор):  8.5872
6) Технологический институт(1,2):  8.0061
7) Пл. Александра Невского (1,2):  6.6677
8) Лиговский проспект:  3.7806
9) Адмиралтейская:  2.7308
10) Обводный канал:  2.6112

Сред